In [ ]:
!pip install absl-py==2.1.0 astunparse==1.6.3 certifi==2024.12.14 charset-normalizer==3.4.0 flatbuffers==24.3.25 gast==0.6.0 google-pasta==0.2.0 grpcio==1.68.1 h5py==3.12.1 idna==3.10 keras==3.5.0 libclang==18.1.1 Markdown==3.7 markdown-it-py==3.0.0 MarkupSafe==3.0.2 mdurl==0.1.2 ml-dtypes==0.4.1 namex==0.0.8 numpy==1.26.4 opt_einsum==3.4.0 optree==0.13.1 packaging==24.2 protobuf==4.25.5 Pygments==2.18.0 requests==2.32.3 rich==13.9.4 setuptools==75.6.0 six==1.17.0 tensorboard==2.17.1 tensorboard-data-server==0.7.2 tensorflow==2.17.0  termcolor==2.5.0 typing_extensions==4.12.2 urllib3==2.2.3 Werkzeug==3.1.3 wheel==0.45.1 wrapt==1.17.0

In [1]:
!pip install mtcnn[tensorflow]

  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached markdown-3.8.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached n

In [2]:
!pip install opencv-python

  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (67.0 MB)


In [3]:
!pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 2.9 MB/s eta 0:00:003.3 MB/s eta 0:00:01


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications.xception import preprocess_input

# Import MTCNN for face detection
from mtcnn import MTCNN
from PIL import Image

2025-07-29 14:18:05.850666: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753780685.866166  299004 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753780685.870730  299004 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753780685.882606  299004 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753780685.882626  299004 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753780685.882628  299004 computation_placer.cc:177] computation placer alr

In [5]:
import os
import numpy as np
import cv2

In [6]:
# Parameters
TIME_STEPS = 30  # Number of frames per video
HEIGHT, WIDTH = 299, 299

# Paths
model_path = 'backend/ml_models/XceptionNet.keras'  # Update with your model path
# video_path = '/content/drive/MyDrive/Dataset DDM/Saved models_trial 1/demo 2.mp4'  # Update with your video path

In [7]:
def build_model(lstm_hidden_size=256, num_classes=2, dropout_rate=0.5):
    # Input shape: (batch_size, TIME_STEPS, HEIGHT, WIDTH, 3)
    inputs = layers.Input(shape=(TIME_STEPS, HEIGHT, WIDTH, 3))

    # TimeDistributed layer to apply the base model to each frame
    base_model = keras.applications.Xception(weights='imagenet', include_top=False, pooling='avg')
    # For inference, we don't need to set trainable, but if you plan to retrain, you can set accordingly
    # base_model.trainable = False

    # Apply TimeDistributed wrapper
    x = layers.TimeDistributed(base_model)(inputs)
    # x shape: (batch_size, TIME_STEPS, 2048)

    # LSTM layer
    x = layers.LSTM(lstm_hidden_size)(x)

    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs)
    return model

# Load the model architecture
model = build_model()
# Load weights into the model
model.load_weights(model_path)

W0000 00:00:1753780700.068059  299004 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
# --- Image Processing Function (now processes the entire image as a frame) ---
def extract_face_from_image(image_path, target_height=HEIGHT, target_width=WIDTH):
    """
    Loads an image from the given path, resizes it to target dimensions,
    and returns the preprocessed image array.
    Face detection is skipped; the entire image is treated as the input frame.

    Args:
        image_path (str): Path to the input image file.
        target_height (int): Desired height for the processed image.
        target_width (int): Desired width for the processed image.

    Returns:
        numpy.ndarray: Preprocessed image array (HEIGHT, WIDTH, 3) or a black
                       image array if an error occurs during loading.
    """
    try:
        # Load the image using OpenCV
        img = cv2.imread(image_path)
        if img is None:
            raise FileNotFoundError(f"Error: Image not found at {image_path}")
        # Convert BGR (OpenCV default) to RGB (for Keras)
        frame_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    except Exception as e:
        print(f"Error loading image '{image_path}': {e}")
        # Return a black image if loading fails
        return np.zeros((target_height, target_width, 3), dtype=np.float32)

    # Resize the entire image to target dimensions
    # Using PIL for resizing as it's often used with preprocess_input
    image_pil = Image.fromarray(frame_rgb)
    resized_image = image_pil.resize((target_width, target_height))

    # Convert back to numpy array and preprocess for the Xception model
    image_array = np.array(resized_image)
    image_array = preprocess_input(image_array) # Xception specific preprocessing
    return image_array

In [9]:
image_path = 'data/image.png'

In [10]:
# Get the preprocessed single frame from the image file
single_frame_array = extract_face_from_image(image_path, target_height=HEIGHT, target_width=WIDTH)

# Replicate the single preprocessed frame TIME_STEPS (30) times
# This creates a sequence of identical frames: (TIME_STEPS, HEIGHT, WIDTH, 3)
sequence_of_frames = np.array([single_frame_array] * TIME_STEPS)

# Add the batch dimension: (1, TIME_STEPS, HEIGHT, WIDTH, 3)
input_for_prediction = np.expand_dims(sequence_of_frames, axis=0)